In [21]:
import copy
import itertools
import os
from typing import List

import matplotlib.pyplot as plt
import pandas as pd
from pydantic import BaseModel, Field
from transformers import AutoTokenizer

In [22]:
from pydantic_settings import BaseSettings, SettingsConfigDict

class EnvSettings(BaseSettings):
    model_config = SettingsConfigDict(
        env_file="../.env", env_file_encoding="utf-8", extra="ignore"
    )
    embedding_base_url: str
    embedding_api_key: str
    embedding_model: str
    embedding_model_dir: str
    
    sample_data_dir: str
    pipeline_src_dir: str
settings = EnvSettings()

import sys
sys.path.append(settings.pipeline_src_dir)

In [23]:
class DBSettings(BaseSettings):
    model_config = SettingsConfigDict(
        env_file="database/pgvector_llamaindex/.env", env_file_encoding="utf-8", extra="ignore"
    )
    postgres_user: str
    postgres_password: str
    postgres_db: str
    postgres_url: str
    postgres_port: str

db_settings = DBSettings()
print(db_settings.postgres_db)

pgvector_llamaindex


# Prepare Embedder & VectorStore

In [24]:
from llama_index.core import Settings
from llama_index.embeddings.text_embeddings_inference import (
    TextEmbeddingsInference,
)

from llama_index.core import SimpleDirectoryReader, StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.postgres import PGVectorStore

from sqlalchemy import make_url

In [25]:
# https://docs.llamaindex.ai/en/stable/examples/embeddings/text_embedding_inference/
embed_model = TextEmbeddingsInference(
    model_name=settings.embedding_model,
    base_url=settings.embedding_base_url,
    timeout=60,
    embed_batch_size=10,
)
Settings.embed_model = embed_model

In [26]:
import psycopg2
# connection_string = "postgresql://{}:{}@localhost:{}/{}".format(
connection_string = "postgresql://{}:{}@localhost:{}/{}".format(
    db_settings.postgres_user,
    db_settings.postgres_password,
    db_settings.postgres_port,
    db_settings.postgres_db
)

db_name = db_settings.postgres_db
print(f"DB: {db_name}")
conn = psycopg2.connect(connection_string)
conn.autocommit=True

DB: pgvector_llamaindex


In [27]:
## Initialize vector store instance
url = make_url(connection_string)

## hnsw indexing config
hnsw_config = {
    "hnsw_m": 16,
    "hnsw_ef_construction": 64,
    "hnsw_ef_search": 40,
    "hnsw_dist_method": "vector_cosine_ops",
}

## Prepare Multiple vectorstores

In [28]:
vector_store1 = PGVectorStore.from_params(
    database=db_name,
    host=url.host,
    password=url.password,
    port=url.port,
    user=url.username,
    table_name="test_documents",
    embed_dim=1024,  #bge-m3
    hnsw_kwargs=hnsw_config,
)

## create storage context
storage_context1 = StorageContext.from_defaults(vector_store=vector_store1)

In [29]:
vector_store2 = PGVectorStore.from_params(
    database=db_name,
    host=url.host,
    password=url.password,
    port=url.port,
    user=url.username,
    table_name="test_documents2",
    embed_dim=1024,  #bge-m3
    hnsw_kwargs=hnsw_config,
)

## create storage context
storage_context2 = StorageContext.from_defaults(vector_store=vector_store2)

In [30]:
## Insert Documents
from llama_index.core import Document
document1 = Document(
    text = "I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata = {"source": "tweet"},
    text_template='{content}'
)

document2 = Document(
    text = "The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata = {"source": "news"},
    text_template='{content}'
)
documents = [document1, document2]

In [31]:
## Insert to storage1
index1 = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context1, show_progress=True
)

Parsing nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

In [32]:
index1

In [33]:
## Insert to storage2
index2 = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context2, show_progress=True
)
index2

Parsing nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

In [36]:
## Check Retrieval
retriever_args = {"similarity_top_k": 10}
retriever = index1.as_retriever(**retriever_args)
## Simple query
query = "LangChain provides abstractions to make working with LLMs easy"
nodes = retriever.retrieve(query)
nodes

[NodeWithScore(node=TextNode(id_='face84da-6e95-41a0-9500-49125f81ad62', embedding=None, metadata={'source': 'tweet'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='220fc440-7be5-4661-9fbb-1234856a0ee6', node_type='4', metadata={'source': 'tweet'}, hash='c211cf902096529c230ab1394516787d54390c9228e4f376fa12fa2c9699a6d9')}, metadata_template='{key}: {value}', metadata_separator='\n', text='I had chocalate chip pancakes and scrambled eggs for breakfast this morning.', mimetype='text/plain', start_char_idx=0, end_char_idx=76, metadata_seperator='\n', text_template='{content}'), score=0.37611465142790657),
 NodeWithScore(node=TextNode(id_='ff4293ac-9c8c-4fdd-9017-f55af542ff92', embedding=None, metadata={'source': 'news'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='4b31a89b-e5bd-4d08-959c-1cdff8a6dd59', node_typ

In [37]:
## Check Retrieval
retriever_args = {"similarity_top_k": 10}
retriever = index2.as_retriever(**retriever_args)
## Simple query
query = "LangChain provides abstractions to make working with LLMs easy"
nodes = retriever.retrieve(query)
nodes

[NodeWithScore(node=TextNode(id_='526e091b-a15b-4966-9c2e-dd5e29407bf8', embedding=None, metadata={'source': 'tweet'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='220fc440-7be5-4661-9fbb-1234856a0ee6', node_type='4', metadata={'source': 'tweet'}, hash='c211cf902096529c230ab1394516787d54390c9228e4f376fa12fa2c9699a6d9')}, metadata_template='{key}: {value}', metadata_separator='\n', text='I had chocalate chip pancakes and scrambled eggs for breakfast this morning.', mimetype='text/plain', start_char_idx=0, end_char_idx=76, metadata_seperator='\n', text_template='{content}'), score=0.37611465142790657),
 NodeWithScore(node=TextNode(id_='d785bc7e-44b0-495e-b4af-7c274e5c18db', embedding=None, metadata={'source': 'news'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='4b31a89b-e5bd-4d08-959c-1cdff8a6dd59', node_typ

In [34]:
## Check DB Schema
conn = psycopg2.connect(connection_string)
with conn.cursor() as cur:
    # --- Print out all schema names ---
    print("All Schemas in the Database:")
    cur.execute("SELECT schema_name FROM information_schema.schemata;")
    schemas = cur.fetchall()
    for schema in schemas:
        print(f"  - {schema[0]}")
        
    ## Print table schemas
    cur.execute("""
        SELECT table_schema, table_name
        FROM information_schema.tables
        WHERE table_schema NOT IN ('information_schema', 'pg_catalog')
        ORDER BY table_schema, table_name;
    """)
    tables = cur.fetchall()

    # 3. Print the schema (columns) of each table
    for schema_name, table_name in tables:
        print(f"\nTable: {schema_name}.{table_name}")
        
        # Fetch column details from information_schema.columns
        cur.execute("""
            SELECT column_name, data_type, is_nullable, column_default
            FROM information_schema.columns
            WHERE table_schema = %s
            AND table_name   = %s
            ORDER BY ordinal_position;
        """, (schema_name, table_name))
        
        columns = cur.fetchall()
        if not columns:
            print("  (No columns found)")
        else:
            for col_name, col_type, is_nullable, default_val in columns:
                print(f"  - Column: {col_name}")
                print(f"    Data Type: {col_type}")
                print(f"    Nullable:  {is_nullable}")
                print(f"    Default:   {default_val}")

    cur.close()
conn.close()

All Schemas in the Database:
  - public
  - information_schema
  - pg_catalog
  - pg_toast

Table: public.data_test_documents
  - Column: id
    Data Type: bigint
    Nullable:  NO
    Default:   nextval('data_test_documents_id_seq'::regclass)
  - Column: text
    Data Type: character varying
    Nullable:  NO
    Default:   None
  - Column: metadata_
    Data Type: json
    Nullable:  YES
    Default:   None
  - Column: node_id
    Data Type: character varying
    Nullable:  YES
    Default:   None
  - Column: embedding
    Data Type: USER-DEFINED
    Nullable:  YES
    Default:   None

Table: public.data_test_documents2
  - Column: id
    Data Type: bigint
    Nullable:  NO
    Default:   nextval('data_test_documents2_id_seq'::regclass)
  - Column: text
    Data Type: character varying
    Nullable:  NO
    Default:   None
  - Column: metadata_
    Data Type: json
    Nullable:  YES
    Default:   None
  - Column: node_id
    Data Type: character varying
    Nullable:  YES
    Defaul